## Banking Data Cleaning and Feature Engineering

- This notebook loads the raw bank customer dataset from PostgreSQL, applies data cleaning operations (including type conversions and handling of null values), and creating some new calculated columns.

In [1]:
# importing libraries
import pandas as pd
import numpy as np 


## Database Connection

In [2]:
# connecting to the database

%run Ingestion_db.py 

#Loading raw data

df = pd.read_sql("SELECT * FROM bank_customer",engine)
print(df.head)

<bound method NDFrame.head of      Client ID              Name  Age  Location ID Joined Bank  \
0     IND81288     Raymond Mills   24        34324  06-05-2019   
1     IND65833     Julia Spencer   23        42205  10-12-2001   
2     IND47499    Stephen Murray   27         7314  25-01-2010   
3     IND72498    Virginia Garza   40        34594  28-03-2019   
4     IND60181   Melissa Sanders   46        41269  20-07-2012   
...        ...               ...  ...          ...         ...   
2995  IND66827         Earl Hall   82         8760  09-10-2014   
2996  IND40556  Billy Williamson   44        32837  05-02-2009   
2997  IND72414      Victor Black   70        36088  29-12-2009   
2998  IND46652       Andrew Ford   56        24871  13-02-2006   
2999  IND40216        Amy Nguyen   79        38518  08-12-2005   

          Banking Contact Nationality                     Occupation  \
0          Anthony Torres    American           Safety Technician IV   
1        Jonathan Hawkins     Afr

## Data Cleaning and Preparation

In [3]:
df.head()

,Client ID,Name,Age,Location ID,Joined Bank,Banking Contact,Nationality,Occupation,Fee Structure,Loyalty Classification,...,Bank Deposits,Checking Accounts,Saving Accounts,Foreign Currency Account,Business Lending,Properties Owned,Risk Weighting,BRId,GenderId,IAId
0,IND81288,Raymond Mills,24,34324,06-05-2019,Anthony Torres,American,Safety Technician IV,High,Jade,...,1485828.64,603617.88,607332.46,12249.96,1134475.30,1,2,1,1,1
1,IND65833,Julia Spencer,23,42205,10-12-2001,Jonathan Hawkins,African,Software Consultant,High,Jade,...,641482.79,229521.37,344635.16,61162.31,2000526.10,1,3,2,1,2
2,IND47499,Stephen Murray,27,7314,25-01-2010,Anthony Berry,European,Help Desk Operator,High,Gold,...,1033401.59,652674.69,203054.35,79071.78,548137.58,1,3,3,2,3
3,IND72498,Virginia Garza,40,34594,28-03-2019,Steve Diaz,American,Geologist II,Mid,Silver,...,1048157.49,1048157.49,234685.02,57513.65,1148402.29,0,4,4,1,4
4,IND60181,Melissa Sanders,46,41269,20-07-2012,Shawn Long,American,Assistant Professor,Mid,Platinum,...,487782.53,446644.25,128351.45,30012.14,1674412.12,0,3,1,2,5


In [4]:
df.shape

(3000, 25)

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000 entries, 0 to 2999
Data columns (total 25 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Client ID                 3000 non-null   object 
 1   Name                      3000 non-null   object 
 2   Age                       3000 non-null   int64  
 3   Location ID               3000 non-null   int64  
 4   Joined Bank               3000 non-null   object 
 5   Banking Contact           3000 non-null   object 
 6   Nationality               3000 non-null   object 
 7   Occupation                3000 non-null   object 
 8   Fee Structure             3000 non-null   object 
 9   Loyalty Classification    3000 non-null   object 
 10  Estimated Income          3000 non-null   float64
 11  Superannuation Savings    3000 non-null   float64
 12  Amount of Credit Cards    3000 non-null   int64  
 13  Credit Card Balance       3000 non-null   float64
 14  Bank Loa

In [6]:
# checking for null values 

print("Null values counts per column")
df.isnull().sum()

Null values counts per column


Client ID                   0
Name                        0
Age                         0
Location ID                 0
Joined Bank                 0
Banking Contact             0
Nationality                 0
Occupation                  0
Fee Structure               0
Loyalty Classification      0
Estimated Income            0
Superannuation Savings      0
Amount of Credit Cards      0
Credit Card Balance         0
Bank Loans                  0
Bank Deposits               0
Checking Accounts           0
Saving Accounts             0
Foreign Currency Account    0
Business Lending            0
Properties Owned            0
Risk Weighting              0
BRId                        0
GenderId                    0
IAId                        0
dtype: int64

In [7]:
df.columns

Index(['Client ID', 'Name', 'Age', 'Location ID', 'Joined Bank',
       'Banking Contact', 'Nationality', 'Occupation', 'Fee Structure',
       'Loyalty Classification', 'Estimated Income', 'Superannuation Savings',
       'Amount of Credit Cards', 'Credit Card Balance', 'Bank Loans',
       'Bank Deposits', 'Checking Accounts', 'Saving Accounts',
       'Foreign Currency Account', 'Business Lending', 'Properties Owned',
       'Risk Weighting', 'BRId', 'GenderId', 'IAId'],
      dtype='object')

### Deriving new features from existing data to support better analysis and decision-making


In [8]:
# defining band boundaries
bins = [0,100000,300000,float('inf')]

labels =['Low' , 'Mid' , 'High']

df['Income Band'] = pd.cut(df['Estimated Income'],bins =bins,labels = labels , right = False)

In [9]:
df['Income Band'].value_counts()

Income Band
Mid     1517
Low     1027
High     456
Name: count, dtype: int64

In [10]:
# mapping dictionaries 
gender_map = {1: 'Male', 2: 'Female'}

brid_map = {
    1: 'Retail',
    2: 'Institutional',
    3: 'Private Bank',
    4: 'Commercial'
}

iaid_map = {
    1: 'Victor Dean',
    2: 'Jeremy Porter',
    3: 'Ernest Knight',
    4: 'Eric Shaw',
    5: 'Kevin Kim',
    6: 'Victor Rogers',
    7: 'Eugene Cunningham',
    8: 'Joe Carroll',
    9: 'Steve Sanchez',
    10: 'Lawrence Sanchez',
    11: 'Peter Castillo',
    12: 'Victor Gutierrez',
    13: 'Daniel Carroll',
    14: 'Carl Anderson',
    15: 'Nicholas Ward',
    16: 'Fred Bryant',
    17: 'Ryan Taylor',
    18: 'Sean Vasquez',
    19: 'Nicholas Morrison',
    20: 'Jack Phillips',
    21: 'Juan Ramirez',
    22: 'Gregory Boyd'
}

# mapping values to new columns

df['Gender'] = df['GenderId'].map(gender_map)
df['Banking Relationship'] = df['BRId'].map(brid_map)
df['Investment Advisor'] = df['IAId'].map(iaid_map)

In [11]:
df.head()

,Client ID,Name,Age,Location ID,Joined Bank,Banking Contact,Nationality,Occupation,Fee Structure,Loyalty Classification,...,Business Lending,Properties Owned,Risk Weighting,BRId,GenderId,IAId,Income Band,Gender,Banking Relationship,Investment Advisor
0,IND81288,Raymond Mills,24,34324,06-05-2019,Anthony Torres,American,Safety Technician IV,High,Jade,...,1134475.30,1,2,1,1,1,Low,Male,Retail,Victor Dean
1,IND65833,Julia Spencer,23,42205,10-12-2001,Jonathan Hawkins,African,Software Consultant,High,Jade,...,2000526.10,1,3,2,1,2,Mid,Male,Institutional,Jeremy Porter
2,IND47499,Stephen Murray,27,7314,25-01-2010,Anthony Berry,European,Help Desk Operator,High,Gold,...,548137.58,1,3,3,2,3,Mid,Female,Private Bank,Ernest Knight
3,IND72498,Virginia Garza,40,34594,28-03-2019,Steve Diaz,American,Geologist II,Mid,Silver,...,1148402.29,0,4,4,1,4,High,Male,Commercial,Eric Shaw
4,IND60181,Melissa Sanders,46,41269,20-07-2012,Shawn Long,American,Assistant Professor,Mid,Platinum,...,1674412.12,0,3,1,2,5,Mid,Female,Retail,Kevin Kim


In [12]:
# Dropping unnecessary columns
df = df.drop(['BRId' , 'GenderId', 'IAId'] , axis =1)

In [14]:
bins = [17, 22, 35, 45, 55, 65, 100]  
labels = ['17-21', '22-34', '35-44', '45-54', '55-64', '65+']
df['Age Groups'] = pd.cut(df['Age'], bins=bins, labels=labels, right=False)

# Check
print(df['Age Groups'].value_counts())
print(df.groupby('Age Groups', observed = True)['Age'].agg(['min', 'max', 'count']))

Age Groups
65+      915
22-34    578
45-54    459
35-44    425
55-64    420
17-21    203
Name: count, dtype: int64
            min  max  count
Age Groups                 
17-21        17   21    203
22-34        22   34    578
35-44        35   44    425
45-54        45   54    459
55-64        55   64    420
65+          65   85    915


In [15]:
df.columns

Index(['Client ID', 'Name', 'Age', 'Location ID', 'Joined Bank',
       'Banking Contact', 'Nationality', 'Occupation', 'Fee Structure',
       'Loyalty Classification', 'Estimated Income', 'Superannuation Savings',
       'Amount of Credit Cards', 'Credit Card Balance', 'Bank Loans',
       'Bank Deposits', 'Checking Accounts', 'Saving Accounts',
       'Foreign Currency Account', 'Business Lending', 'Properties Owned',
       'Risk Weighting', 'Income Band', 'Gender', 'Banking Relationship',
       'Investment Advisor', 'Age Groups'],
      dtype='object')

In [16]:
# Calculating Estimated Annual Income from Estimated Income to standardize financial metrics

df['Estimated Annual Income'] = df['Estimated Income']*12

- In this analysis, the `Properties Owned` column represents the number of properties held by each customer. Since the dataset does not provide actual property values, including this column without assumptions could lead to **unrealistic Net Worth calculations**. 

In [17]:
Avg_property_value = 4000000

df['Property Value'] = df['Properties Owned']*Avg_property_value

## Banking Analytics Metrics

- **Net Worth:** Total assets minus total liabilities to assess overall financial health.  
- **Debt-to-Income (DTI) Ratio:** Total liabilities divided by estimated annual income to evaluate debt burden.  
 



In [18]:
# Calcualting Assets of each customer

df['Assets'] = df['Bank Deposits']+df['Checking Accounts']+df['Saving Accounts']+df['Foreign Currency Account']+df['Superannuation Savings']+df['Property Value']

In [19]:
df['Liablities'] =  df['Bank Loans']+df['Credit Card Balance']+df['Business Lending']

In [20]:
df['Net Worth'] = df['Assets'] - df['Liablities']

In [21]:
df['Debt to Income Ratio']=df['Liablities']/df['Estimated Annual Income']

In [22]:
df.head()

,Client ID,Name,Age,Location ID,Joined Bank,Banking Contact,Nationality,Occupation,Fee Structure,Loyalty Classification,...,Gender,Banking Relationship,Investment Advisor,Age Groups,Estimated Annual Income,Property Value,Assets,Liablities,Net Worth,Debt to Income Ratio
0,IND81288,Raymond Mills,24,34324,06-05-2019,Anthony Torres,American,Safety Technician IV,High,Jade,...,Male,Retail,Victor Dean,22-34,904617.24,4000000,6726706.89,1911202.76,4815504.13,2.112720
1,IND65833,Julia Spencer,23,42205,10-12-2001,Jonathan Hawkins,African,Software Consultant,High,Jade,...,Male,Institutional,Jeremy Porter,22-34,3478011.72,4000000,5294200.55,3273398.41,2020802.14,0.941169
2,IND47499,Stephen Murray,27,7314,25-01-2010,Anthony Berry,European,Help Desk Operator,High,Gold,...,Female,Private Bank,Ernest Knight,22-34,2039222.76,4000000,6011028.31,1605422.16,4405606.15,0.787272
3,IND72498,Virginia Garza,40,34594,28-03-2019,Steve Diaz,American,Geologist II,Mid,Silver,...,Male,Commercial,Eric Shaw,35-44,4281697.32,0,2393986.80,1273802.35,1120184.45,0.297499
4,IND60181,Melissa Sanders,46,41269,20-07-2012,Shawn Long,American,Assistant Professor,Mid,Platinum,...,Female,Retail,Kevin Kim,45-54,1568540.16,0,1140867.97,2726493.56,-1585625.59,1.738236


In [23]:
df.columns

Index(['Client ID', 'Name', 'Age', 'Location ID', 'Joined Bank',
       'Banking Contact', 'Nationality', 'Occupation', 'Fee Structure',
       'Loyalty Classification', 'Estimated Income', 'Superannuation Savings',
       'Amount of Credit Cards', 'Credit Card Balance', 'Bank Loans',
       'Bank Deposits', 'Checking Accounts', 'Saving Accounts',
       'Foreign Currency Account', 'Business Lending', 'Properties Owned',
       'Risk Weighting', 'Income Band', 'Gender', 'Banking Relationship',
       'Investment Advisor', 'Age Groups', 'Estimated Annual Income',
       'Property Value', 'Assets', 'Liablities', 'Net Worth',
       'Debt to Income Ratio'],
      dtype='object')

In [24]:

table_name = 'clean_bank_customers'  

df.to_sql(
    name=table_name,
    con=engine,
    if_exists='replace',  
    index=False,  
)

print(f"Cleaned data saved to table '{table_name}' with {len(df)} rows.")

Cleaned data saved to table 'clean_bank_customers' with 3000 rows.


In [26]:
csv_path = 'clean_bank_customers.csv'

df.to_csv(csv_path,index = False)